In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [ ]:
indicators = pd.read_parquet("../../data/indicators/US/all_indicators_raw_outer.parquet", engine="pyarrow")
indicators["date"] = pd.to_datetime(indicators["date"])
indicators.reset_index(drop=True, inplace=True)

In [ ]:
nber_recessions = pd.read_parquet("../../data/indicators/US/nber_recession.parquet")
nber_recessions["date"] = pd.to_datetime(nber_recessions["date"])
nber_recessions = nber_recessions[nber_recessions["date"] >= "1962-01-01"]

In [ ]:
us_top_500 = pd.read_parquet("../../data/indicators/US/us_top_500.parquet", engine="pyarrow")
us_top_500["date"] = pd.to_datetime(us_top_500["date"])
data = pd.merge(indicators, us_top_500, on=["date"], how="outer")

In [ ]:
data.set_index("date", inplace=True)
np.random.seed(49)

In [ ]:
epu = pd.read_excel("../../data/indicators/US/US_Policy_Uncertainty_Data_EPU.xlsx")
epu = epu[["Date", "Three_Component_Index", "News_Based_Policy_Uncert_Index"]]
epu.rename(columns={"Date": "date", "Three_Component_Index" : "index", "News_Based_Policy_Uncert_Index" : "news_index"}, inplace=True)
epu["date"] = pd.to_datetime(epu["date"])
epu.set_index("date", inplace=True)
epu.dropna(inplace=True)

In [ ]:
epu.tail()

## Plotting the dates calculated below

In [ ]:
epu_rec_dates = pd.read_csv("../../time_periods/model_train_ready/EPU_rec_2yr.csv")
epu_rec_dates["date"] = pd.to_datetime(epu_rec_dates["date"])

In [ ]:
epu_rec_dates = pd.read_csv("../../time_periods/model_train_ready/EPU_rec_5yr.csv")
epu_rec_dates["date"] = pd.to_datetime(epu_rec_dates["date"])

In [ ]:
epu_exp_dates = pd.read_csv("../../time_periods/model_train_ready/EPU_exp_2yr.csv")
epu_exp_dates["date"] = pd.to_datetime(epu_exp_dates["date"])

In [ ]:
epu_exp_dates

In [ ]:
nber_rec_dates = pd.read_csv("../../time_periods/model_train_ready/nber_recession_dates.csv")
nber_rec_dates["date"] = pd.to_datetime(nber_rec_dates["date"])


In [ ]:
#One algo

fig, ax = plt.subplots(1, figsize=(12, 4), sharex=False)

data_copy = data.copy()

min_date = pd.Timestamp("1985-01-01")

resample_freq = "MS"


market_cap = data_copy[data_copy.index > min_date]["market_cap_usd"].resample(resample_freq).first()

ax2 = ax.twinx()

ax2.plot(market_cap, alpha=0.5, color="tab:orange", linewidth=2)
ax2.set_yscale('log')

current_i = 0
for i in range(len(epu_rec_dates['date'])-1):
    if epu_rec_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == epu_rec_dates['date'].iloc[i]:
        continue
    ax.axvspan(epu_rec_dates['date'].iloc[current_i], epu_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='red', alpha=0.3, ymin=0.5, ymax=0.9)
    current_i = i + 1
ax.axvspan(epu_rec_dates['date'].iloc[current_i], epu_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='red', alpha=0.3, ymin=0.5, ymax=0.9)

    
current_i = 0
for i in range(len(epu_exp_dates['date'])-1):
    if epu_exp_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == epu_exp_dates['date'].iloc[i]:
        continue
    ax.axvspan(epu_exp_dates['date'].iloc[current_i], epu_exp_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='blue', alpha=0.3, ymin=0.1, ymax=0.5)
    current_i = i + 1
ax.axvspan(epu_exp_dates['date'].iloc[current_i], epu_exp_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='blue', alpha=0.3, ymin=0.1, ymax=0.5)

ax.plot(epu["index"], label="EPU")

current_i = 0
for i in range(len(nber_rec_dates['date'])-1):
    if nber_rec_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == nber_rec_dates['date'].iloc[i]:
        continue
    ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)
    current_i = i + 1
ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)

rolling_avg = epu["index"].rolling(f'{5*12*30}D').mean()
criterion = (epu["index"] > rolling_avg)

ax.tick_params(axis='both', which='major', labelsize=14)

ax2.axes.get_yaxis().set_ticks([])

ax.set_ylabel("EPU Index", fontsize=16)

plt.xlim(min_date, pd.Timestamp("2023-12-31"))
plt.tight_layout()

In [ ]:
fig.savefig("../../figures/epu-filter-rec-and-exp-2yr.pdf", dpi=3000)

## USing the filter, saving the dates

In [ ]:
epu["class"] = 0
epu.loc[epu["index"] < epu["index"].rolling(f'{2*12*30}D').mean(), "class"] = 1
epu_daily = epu.resample("D").ffill()


In [ ]:
pd.DataFrame(epu_daily[epu_daily["class"] == 1].index, columns=["date"]).to_csv("../../time_periods/model_train_ready/EPU_exp_2yr.csv", index=False)

In [ ]:
epu["class"] = 0
epu.loc[(epu["index"] > epu["index"].rolling(f'{2*12*30}D').mean()), "class"] = 1
epu_daily = epu.resample("D").ffill()

In [ ]:
pd.DataFrame(epu_daily[epu_daily["class"] == 1].index, columns=["date"]).to_csv("../../time_periods/model_train_ready/EPU_exp_2yr.csv", index=False)